In [1]:
import re
import numpy as np
import pandas as pd

In [2]:
source_dir = '../data/brologs/03-22-13/'
#source_dir = '../data/brologs/03-28/'

In [3]:
df_conn = pd.read_json(source_dir + 'conn.log', lines=True)
df_conn.head()

,conn_state,duration,history,id.orig_h,id.orig_p,id.resp_h,id.resp_p,missed_bytes,orig_bytes,orig_ip_bytes,orig_pkts,proto,resp_bytes,resp_ip_bytes,resp_pkts,service,ts,tunnel_parents,uid
0,S0,NaN,D,192.168.3.183,37942,8.8.8.8,53,0,NaN,67,1,udp,NaN,0,0,dns,1.521699e+09,[],Cbzg0v4gAOcdQDT7vk
1,S0,NaN,S,192.168.3.183,41773,212.69.166.138,80,0,NaN,60,1,tcp,NaN,0,0,NaN,1.521699e+09,[],CRWrOH2KMH49kJkDFk
2,SF,0.000448,Dd,192.168.3.183,45880,172.16.3.20,53,0,39.0,67,1,udp,103.0,131,1,dns,1.521699e+09,[],CaOSEj1M9W3hMahyB4
3,S0,NaN,D,192.168.3.183,34280,8.8.8.8,53,0,NaN,63,1,udp,NaN,0,0,dns,1.521699e+09,[],CFgQI52Aeslpplsp8j
4,SF,0.014634,ShADdafF,192.168.3.61,62015,172.16.1.58,1433,0,1234.0,1606,9,tcp,735.0,1067,8,NaN,1.521699e+09,[],CBfLai249qVUc0iDze


In [4]:
df_tds = pd.read_json(source_dir + 'tds.log', lines=True)
df_tds.head()

,id.orig_h,id.orig_p,id.resp_h,id.resp_p,tdstype,ts,uid
0,192.168.3.61,61617,172.16.1.58,1433,14,1.521699e+09,CE6EJo3e4GljoEcib8
1,192.168.3.61,61617,172.16.1.58,1433,1,1.521699e+09,CE6EJo3e4GljoEcib8
2,192.168.3.61,61617,172.16.1.58,1433,14,1.521699e+09,CE6EJo3e4GljoEcib8
3,192.168.3.63,57093,192.168.3.61,1433,1,1.521699e+09,CMvkHk4ZwzjDaI6ozi
4,192.168.3.63,57093,192.168.3.61,1433,3,1.521699e+09,CMvkHk4ZwzjDaI6ozi


In [9]:
df_tds.groupby('id.orig_h')['uid'].nunique()

id.orig_h
192.168.3.61    42
192.168.3.62     1
192.168.3.63     8
Name: uid, dtype: int64

In [5]:
df_tds_sqlbatch = pd.read_json(source_dir + 'tds_sqlbatch.log', lines=True)
df_tds_sqlbatch.head()

,id.orig_h,id.orig_p,id.resp_h,id.resp_p,sqlbatch,ts,uid
0,192.168.3.61,61617,172.16.1.58,1433,select * from EXT_ORDER_NEW\r\nWhere UPDSTATUS...,1.521699e+09,CE6EJo3e4GljoEcib8
1,192.168.3.63,57093,192.168.3.61,1433,set implicit_transactions on,1.521699e+09,CMvkHk4ZwzjDaI6ozi
2,192.168.3.61,61617,172.16.1.58,1433,select * from EXT_ORDER_NEW\r\nWhere UPDSTATUS...,1.521699e+09,CE6EJo3e4GljoEcib8
3,192.168.3.61,61617,172.16.1.58,1433,select * from EXT_ORDER_NEW\r\nWhere UPDSTATUS...,1.521699e+09,CE6EJo3e4GljoEcib8
4,192.168.3.61,61617,172.16.1.58,1433,select * from EXT_ORDER_NEW\r\nWhere UPDSTATUS...,1.521699e+09,CE6EJo3e4GljoEcib8


In [11]:
df_tds_sqlbatch.groupby('uid')['id.orig_h'].nunique()

uid
CE6EJo3e4GljoEcib8    1
CMvkHk4ZwzjDaI6ozi    1
CvZ9IM3yeAwuILO609    1
Name: id.orig_h, dtype: int64

In [6]:
df_tds_sqlbatch.describe()

,id.orig_p,id.resp_p,ts
count,2026.000000,2026.0,2.026000e+03
mean,60649.353406,1433.0,1.521700e+09
std,3344.194685,0.0,5.634117e+02
min,49079.000000,1433.0,1.521699e+09
25%,61617.000000,1433.0,1.521700e+09
50%,61617.000000,1433.0,1.521700e+09
75%,61617.000000,1433.0,1.521701e+09
max,61617.000000,1433.0,1.521701e+09


In [31]:
# need to make sure if "all" SQL do not care about the lower case or upper case
def getPII_colnames(sqlquery):
    #print(sqlquery.lower())
    pattern_list = [
        "(?<=select )(.*?)(?= from)", # case 1, between select and from
        "(?<=where )(.*?)(?=\>|\<|\=|between|like|in|is)", # case 2, where a > a0 and/or b > b0, https://www.w3schools.com/sql/sql_where.asp
        "(?<=and )(.*?)(?=\>|\<|\=|between|like|in|is)", # case 2, where a > a0 and/or b > b0, https://www.w3schools.com/sql/sql_where.asp
        "(?<=or )(.*?)(?=\>|\<|\=|between|like|in|is)", # case 2, where a > a0 and/or b > b0, https://www.w3schools.com/sql/sql_where.asp
    ]
    colnames = []
    for pattern in pattern_list:
        thiscolnames = [x.split(',') for x in re.findall(pattern, sqlquery)]
        
        for x in thiscolnames:
            colnames += x
        
    #post polishing... 
    #strip space...
    #remove wildcard???
    colnames = [x.strip(' ') for x in colnames]
    colnames = [x.split('.')[-1] for x in colnames]
    #return colnames
    return list(set(colnames))

def getPII_tabnames(sqlquery):
    # case 1, after from and before space or next line
    pattern_list = [
        "(?<=from )(.*?)(?=\r| |\;)", # case 1, after from and before space or next line
        "(?<=join )(.*?)(?=on)", # case 2, join table on...
    ]
    tabnames = []
    for pattern in pattern_list:
        tabnames += re.findall(pattern, sqlquery)
        
    #post polishing... 
    #strip space...
    tabnames = [x.strip(' ') for x in tabnames]
    return tabnames

testsqlquery = "SELECT Orders.OrderID, Customers.CustomerName, Orders.OrderDate FROM Orders INNER JOIN Customers ON Orders.CustomerID=Customers.CustomerID;"
#testsqlquery = "select * from EXT_ORDER_NEW\u000d\u000aWhere UPDSTATUS IS NULL"
#testsqlquery = "select orderid, orderdate, ordertime, stockcode, barcode, bs, acctype, ordertype, oqty, oprice, reqtype, updstatus, processtime, oldorderid, reqid, accno, modda, modui, insda, insui from EXT_ORDER_NEW;"
#testsqlquery = "select orderid, stockcode, barcode, bs, acctype, ordertype, oqty, oprice, reqtype, updstatus, processtime, oldorderid, reqid, accno, modda, modui, insda, insui from EXT_ORDER_NEW Where UPDSTATUS IS NULL where orderdate > yyyy-mm-dd and ordertime > hh:mm:ss"

print(getPII_colnames(testsqlquery.lower()))
print(getPII_tabnames(testsqlquery.lower()))

['customername', 'orderid', 'orderdate']
['orders', 'customers']
